# News

In [1]:
!pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 
!pip install --upgrade google-auth-oauthlib 
!pip install youtube-transcript-api

In [2]:
import re
import html
from googleapiclient.discovery import build
import os
import pandas as pd
from datetime import datetime, timedelta
import pytz

## Channel Definitions and Setups

In [3]:
# Channels 
CNBC_TV = "UCrp_UI8XtuYfpiqluWLD7Lw"
MY_TIMEZONE='America/Chicago'

In [4]:
# Youtube Data API v3
api_key = os.getenv('YOUTUBE_API_KEY')
if not api_key:
    raise ValueError("No API key found. Make sure the YOUTUBE_API_KEY environment variable is set.")

youtube = build('youtube', 'v3', developerKey=api_key)

## Utils

In [5]:
# Get date ranges (timezone aware)
from datetime import datetime, timedelta
import pytz

def get_date_range(period_type, number=1):
    local_timezone = pytz.timezone(MY_TIMEZONE)
    now = datetime.now(pytz.utc).astimezone(local_timezone)    
    
    if period_type == 'today':
        start_date = datetime(now.year, now.month, now.day, 0, 0, 0)
        end_date = datetime(now.year, now.month, now.day, 23, 59, 59, 999999)
    elif period_type == 'days':
        start_date = datetime(now.year, now.month, now.day, 0, 0, 0) - timedelta(days=number-1)
        end_date = datetime(now.year, now.month, now.day, 23, 59, 59, 999999)
    elif period_type == 'weeks':
        start_date = datetime(now.year, now.month, now.day, 0, 0, 0) - timedelta(weeks=number)
        end_date = datetime(now.year, now.month, now.day, 23, 59, 59, 999999)
    elif period_type == 'months':
        start_date = datetime(now.year, now.month, now.day, 0, 0, 0) - timedelta(days=30*number)
        end_date = datetime(now.year, now.month, now.day, 23, 59, 59, 999999)
    else:
        raise ValueError("Unsupported period type specified.")
    
    # Localize the datetime objects
    start_date = local_timezone.localize(start_date)
    end_date = local_timezone.localize(end_date)
    
    return start_date, end_date

# # Test get_date_range
# start_date, end_date = get_date_range('days',3)
# print(start_date, end_date)

def iso_duration_to_minutes(iso_duration):
    # Parse ISO 8601 duration format to total minutes
    pattern = re.compile(r'PT((?P<hours>\d+)H)?((?P<minutes>\d+)M)?((?P<seconds>\d+)S)?')
    matches = pattern.match(iso_duration)
    if not matches:
        return 0  # Return 0 if the pattern does not match

    hours = int(matches.group('hours') or 0)
    minutes = int(matches.group('minutes') or 0)
    seconds = int(matches.group('seconds') or 0)

    # Calculate total minutes, rounding up if there are any seconds
    total_minutes = hours * 60 + minutes
    if seconds > 0:
        total_minutes += 1  # Round up if there are any remaining seconds

    return total_minutes

# Get formated date
def get_formated_date_today():
    timezone = pytz.timezone(MY_TIMEZONE)
    now = datetime.now(timezone)
    formatted_date = now.strftime('%Y-%m-%d')
    return formatted_date

In [6]:
def fetch_videos(start_date, end_date, channel_id):
    video_data = []
    page_token = None
    local_timezone = pytz.timezone('America/Chicago')  # Define the local timezone

    while True:
        request = youtube.search().list(
            part="snippet",
            channelId=channel_id,
            publishedAfter=start_date.isoformat(),
            publishedBefore=end_date.isoformat(),
            maxResults=50,
            pageToken=page_token,
            type="video"
        )
        response = request.execute()

        video_ids = [item['id']['videoId'] for item in response.get("items", []) if 'videoId' in item['id']]
        if video_ids:
            video_request = youtube.videos().list(
                part='contentDetails, snippet',
                id=','.join(video_ids)
            )
            video_details_response = video_request.execute()

            for video in video_details_response.get("items", []):
                video_id = video['id']  # Ensure video_id is defined here
                content_details = video['contentDetails']
                snippet = video['snippet']
                video_data.append({
                    "Title": html.unescape(snippet['title']),
                    "Published At": datetime.strptime(snippet['publishedAt'], '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=pytz.UTC),
                    "Duration (Min)": iso_duration_to_minutes(content_details.get('duration', 'PT0S')),
                    "Video ID":video_id,
                    "URL": f"https://www.youtube.com/watch?v={video_id}"
                })

        page_token = response.get('nextPageToken')
        if not page_token:
            break

    df = pd.DataFrame(video_data)
    if not df.empty:
        df_sorted = df.sort_values(by=['Published At'])
        return df_sorted
    else:
        print("No videos found.")
        return pd.DataFrame()  # Return an empty DataFrame if no videos were added

In [7]:
import pandas as pd

def filter_videos_by_date_and_time(df_videos, period_type='today', number=1, hour_range=None):
    """
    Filters videos based on the 'Published At' column within the specified date and optional time range.

    Parameters:
    - df_videos (DataFrame): DataFrame containing video data with 'Published At' as datetime.
    - period_type (str): 'today', 'days', 'weeks', or 'months'.
    - number (int): Number of days, weeks, or months to look back from today.
    - hour_range (str, optional): Hour range in the format 'HH-HH'. Only applicable if period_type is 'today'.

    Returns:
    - DataFrame: Filtered DataFrame based on the date and optional time range.
    """
    # Use the existing function to get the date range
    start_date, end_date = get_date_range(period_type, number)

    # Ensure datetime is timezone-aware for comparison
    local_timezone = pytz.timezone(MY_TIMEZONE)
    df_videos['Published At'] = pd.to_datetime(df_videos['Published At']).dt.tz_convert(local_timezone)
    start_date = start_date.astimezone(local_timezone)
    end_date = end_date.astimezone(local_timezone)

    # Filter the DataFrame based on the date range
    filtered_df = df_videos[(df_videos['Published At'] >= start_date) & (df_videos['Published At'] <= end_date)]

    # Apply additional filtering by hour range if specified and period_type is 'today'
    if period_type == 'today' and hour_range:
        start_hour, end_hour = map(int, hour_range.split('-'))
        filtered_df = filtered_df[filtered_df['Published At'].dt.hour >= start_hour]
        filtered_df = filtered_df[filtered_df['Published At'].dt.hour < end_hour]

    return filtered_df

In [8]:
from IPython.display import HTML

def make_clickable(title, url):
    """
    Generates an HTML link for the given title and URL.

    Parameters:
    - title (str): The text to display.
    - url (str): The URL to link to.

    Returns:
    - str: HTML link.
    """
    return f'<a href="{url}" target="_blank">{title}</a>'

def display_df(df):
    """
    Displays the DataFrame in a Jupyter Notebook with clickable titles and without the URL column.

    Parameters:
    - df (DataFrame): DataFrame to display.
    """
    # Create a copy for display to avoid altering the original DataFrame
    df_display = df.copy()
    
    # Make titles clickable
    df_display['Title'] = df_display.apply(lambda x: make_clickable(x['Title'], x['URL']), axis=1)

    # Generate HTML content without the URL column
    html_content = df_display.drop('URL', axis=1).to_html(escape=False, index=False)

    # Display the DataFrame using HTML in Jupyter Notebook
    display(HTML(html_content))

def save_df_to_html(df, file_name):
    """
    Saves the DataFrame to an HTML file with clickable titles and without the URL column.

    Parameters:
    - df (DataFrame): DataFrame to save.
    - file_name (str): Name of the HTML file to save the DataFrame.
    """
    html_content = get_html_content(df)
    with open(file_name, 'w') as file:
        file.write(html_content)
    print(f"DataFrame has been saved to {file_name}.")

def get_html_content(df):
    """
    Generates and returns the HTML content of the DataFrame with clickable titles and without the URL column.

    Parameters:
    - df (DataFrame): DataFrame to generate HTML content from.

    Returns:
    - str: HTML representation of the DataFrame.
    """
    # Create a copy for display to avoid altering the original DataFrame
    df_display = df.copy()
    
    # Make titles clickable
    df_display['Title'] = df_display.apply(lambda x: make_clickable(x['Title'], x['URL']), axis=1)

    # Generate HTML content without the URL column
    html_content = df_display.drop('URL', axis=1).to_html(escape=False, index=False)
    
    return html_content

## Get videos from Youtube CNBC TV Channel

### Get Today's videos

In [24]:
## Getting the videos from the CNBC TV channel
period_type = 'today'  # 'today', 'days', 'weeks', 'months'
number = 2  # The 'today' setting does not use 'number', adjust if using other settings
channel_id = CNBC_TV  # Channel ID as a parameter
start_date, end_date = get_date_range(period_type, number)
df_videos_today = fetch_videos(start_date, end_date, channel_id)

### Filter Today's videos by hour

In [23]:
# Assume df_videos is your initial DataFrame loaded with video data
hour_range1 = '0-8'
hour_range2 = '8-10'
hour_range3 = '10-12'
hour_range4 = '12-14'
hour_range5 = '14-16'
hour_range6 = '16-24'

# Filter videos from the last 3 days
filtered_df = filter_videos_by_date_and_time(df_videos_today, 'today', 1)
display_df(filtered_df)

Title,Published At,Duration (Min),Video ID


## Get transcripts for each video

In [11]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

def get_transcript(video_id):
    """
    Fetches the transcript for a given YouTube video ID and converts it to lowercase.

    Parameters:
    - video_id (str): The YouTube video ID.

    Returns:
    - str: The lowercase transcript text or None if an error occurs.
    """
    try:
        # Fetch the transcript
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        
        # Find the English transcript
        transcript = transcript_list.find_transcript(['en'])
        
        # Fetch the actual transcript
        transcript_data = transcript.fetch()
        
        # Extract the text, combine into sentences, and convert to lowercase
        transcript_text = " ".join([entry['text'] for entry in transcript_data]).lower()
        
        return transcript_text
    
    except Exception as e:
        print(f"An error occurred for video ID {video_id}: {e}")
        return None

def add_transcripts_to_df(df):
    """
    Iterates through the DataFrame, retrieves the transcript for each Video ID,
    and saves the transcript in a new column in the DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the video data.

    Returns:
    - pd.DataFrame: The DataFrame with an additional 'Transcript' column.
    """
    transcripts = []
    for video_id in df['Video ID']:
        transcript = get_transcript(video_id)
        if transcript is None:
            transcript = "No transcript for video"
        transcripts.append(transcript)
    
    df['Transcript'] = transcripts
    return df

In [12]:
# Add transcripts to the DataFrame
df_videos_with_transcripts = add_transcripts_to_df(filtered_df)

An error occurred for video ID UZGWqZKGPTA: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=UZGWqZKGPTA! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
An error occurred for video ID CPevmHflyA4: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=CPevmHflyA4! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/you

In [13]:
display_df(df_videos_with_transcripts)

In [19]:
save_df_to_html(df_videos_with_transcripts, 'videos_with_transcripts.html')

DataFrame has been saved to videos_with_transcripts.html.


## Send emails

In [18]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd

email_user = os.getenv('EMAIL_USER')
email_password = os.getenv('EMAIL_PASSWORD')
email_send1 = 'sliu810@gmail.com'
email_send2 = 'zhengwang827@gmail.com'

recipients = [email_send1]
# Set up the MIME
message = MIMEMultipart()
message['From'] = email_user
message['To'] = email_send1
message['Subject'] = f'CNBC_Videos_{get_formated_date_today()}'

html_content= get_html_content(df_videos_with_transcripts)

# Attach the HTML content to the email
message.attach(MIMEText(html_content, 'html'))

# Function to send email
def send_email():
    try:
        server = smtplib.SMTP('smtp-mail.outlook.com', 587)  # Outlook SMTP server
        #server.set_debuglevel(1)  # Enable debugging output
        server.starttls()
        server.login(email_user, email_password)
        text = message.as_string()
        server.sendmail(email_user, recipients, text)
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Send the email
send_email()

Email sent successfully!
